In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os
os.chdir('/home/sarvagya/RAG_Agents')

from Intelligence.utils.llm_utils import Settings

/home/sarvagya/.cache/pypoetry/virtualenvs/rag-agents-k9FEVAJG-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip list

Package                                  Version
---------------------------------------- -----------
accelerate                               0.30.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
annotated-types                          0.6.0
anyio                                    4.3.0
asgiref                                  3.8.1
asttokens                                2.4.1
async-timeout                            4.0.3
attrs                                    23.2.0
backoff                                  2.2.1
bcrypt                                   4.1.3
beautifulsoup4                           4.12.3
blis                                     0.7.11
boto3                                    1.34.113
botocore                                 1.34.113
Brotli                                   1.1.0
build                                    1.2.1
cachetools                               5.3.3
catalogue                                2

In [3]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-06-05 13:17:49--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  2.60MB/s    in 0.5s    

2024-06-05 13:17:50 (2.60 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-06-05 13:17:50--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [4]:
# march_2022 = SimpleDirectoryReader(
#     input_files=["./data/10q/uber_10q_march_2022.pdf"]
# ).load_data()
# june_2022 = SimpleDirectoryReader(
#     input_files=["./data/10q/uber_10q_june_2022.pdf"]
# ).load_data()
# sept_2022 = SimpleDirectoryReader(
#     input_files=["./data/10q/uber_10q_sept_2022.pdf"]
# ).load_data()

In [5]:
def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=Settings.llm)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

In [6]:
march_tool = get_tool("march_2022", "March 2022")
june_tool = get_tool("june_2022", "June 2022")
sept_tool = get_tool("sept_2022", "September 2022")

In [7]:
query_engine_tools = [march_tool, june_tool, sept_tool]

In [8]:
from llama_index.core.agent import AgentRunner, ReActAgent
# from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker

# openai_step_engine = OpenAIAgentWorker.from_tools(
#     query_engine_tools, llm=llm, verbose=True
# )
# agent = AgentRunner(openai_step_engine)
# # alternative
# agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

# agent_llm = OpenAI(model="gpt-3.5-turbo")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=Settings.llm, verbose=True, max_iterations=20
)

inside ReActAgent, agent_worker = <llama_index.core.agent.react.step.ReActAgentWorker object at 0x7f9cbd4639a0>


In [9]:
# response = agent.chat("Analyze the changes in R&D expenditures and revenue")

In [10]:
# print(str(response))

In [11]:
from pprint import pprint

##### Test Step-Wise Execution

In [12]:
# start task
task = agent.create_task("Analyze the changes in R&D expenditures and revenue")
pprint(task)

Task(task_id='f2e1d9b0-b72f-4e3f-90b4-734bac25e564', input='Analyze the changes in R&D expenditures and revenue', memory=ChatMemoryBuffer(token_limit=24576, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history'), callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f9c86dc4610>, extra_state={'sources': [], 'current_reasoning': [], 'new_memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')})


In [13]:
task.task_id

'f2e1d9b0-b72f-4e3f-90b4-734bac25e564'

In [14]:
agent.agent_worker

In [15]:
step_output = agent.run_step(task.task_id)

print(step_output)

Chatting
input_chat: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: sep

In [16]:
# step_output = agent.run_step(task.task_id, input="What about June?")

In [17]:
s = ''''You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: sept_2022\nTool Description: Provides information about Uber quarterly financials ending September 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the question.\nAction: tool name (one of march_2022, june_2022, sept_2022) if using a tool.\nAction Input: the input to the tool, in a JSON format representing the kwargs (e.g. {"input": "hello world", "num_beams": 5})\n```\n\nPlease ALWAYS start with a Thought.\n\nPlease use a valid JSON format for the Action Input. Do NOT do this {\'input\': \'hello world\', \'num_beams\': 5}.\n\nIf this format is used, the user will respond in the following format:\n\n```\nObservation: tool response\n```\n\nYou should keep repeating the above format till you have enough information to answer the question without using any more tools. At that point, you MUST respond in the one of the following two formats:\n\n```\nThought: I can answer without using any more tools. I\'ll use the user\'s language to answer\nAnswer: [your answer here (In the same language as the user\'s question)]\n```\n\n```\nThought: I cannot answer the question with the provided tools.\nAnswer: [your answer here (In the same language as the user\'s question)]\n```\n\n## Current Conversation\n\nBelow is the current conversation consisting of interleaving human and assistant messages.\n', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Analyze the changes in R&D expenditures and revenue', additional_kwargs={})] 
'''
print(s)

'You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: march_2022
Tool Description: Provides information about Uber quarterly financials ending March 2022
Tool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}

> Tool Name: june_2022
Tool Description: Provides information about Uber quarterly financials ending June 2022
Tool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}

> Tool Name: sept_2022
Tool Description: Provides information about Uber quarterly financials ending September 

In [18]:
response = agent.finalize_response(task.task_id)

In [19]:
response

AgentChatResponse(response="Uber's R&D expenditures have been increasing steadily over the past three quarters, from $492 million in March 2022 to $572 million in September 2022. This represents an increase of approximately 16% over this period. At the same time, Uber's revenue has also been increasing, from $2,587 million in March 2022 to $2,828 million in September 2022, an increase of approximately 9%.", sources=[], source_nodes=[], is_dummy_stream=False)